In [23]:
import pandas as pd
import os
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import r2_score
import featuretools as ft
import time
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectPercentile, f_regression
from sklearn.feature_selection import RFE
import warnings
warnings.filterwarnings('ignore')

In [24]:
start = time.time()

# Read meta data
meta = pd.read_csv('/Users/t.wang/Desktop/Dissertation/Python/input/meta_open.csv', 
                   index_col='uid', parse_dates=['dataend','datastart'], dayfirst=True)#The data will be messed up withou specifying dayfirst


# Read energy data
temporal = pd.read_csv('/Users/t.wang/Desktop/Dissertation/Python/input/temp_open_utc_complete.csv', 
                   index_col='timestamp', parse_dates=True)#.tz_localize('utc')

def loopModels_and_Metrics(ml_Models_names, ml_Models, weatherPoints, n_timeSeriesSplits, 
                           buildingNames, agg_primitives, trans_primitives, varianceThreshold, SelectPercentile_num,
                           SelectPercentile_func, RFE_step):  
    print('\n\n' + ml_Models_names + '\n_____________')
    buildingindex = 0
    for single_building in buildingNames:
        buildingindex+=1
        print('Modelling:' + single_building)
        
        # Read energy data for each given buildingname
        single_timezone = meta.T[single_building].timezone
        startdate = meta.T[single_building].datastart
        enddate = meta.T[single_building].dataend
        single_building_energy = temporal[single_building].tz_convert(single_timezone).truncate(before = startdate, 
                                                            after = enddate)#.fillna(method='bfill').fillna(method='ffill')
                                                            # single_building_energy, some missing data


        # Get weather data for given building
        weatherfile_name = meta.T[single_building].newweatherfilename
        weather_data = pd.read_csv(os.path.join('/Users/t.wang/Desktop/Dissertation/Python/input/',
                                                weatherfile_name),index_col='timestamp', parse_dates=True, na_values='-9999')
        weather_data = weather_data.tz_localize(single_timezone, ambiguous = 'infer')
        weather_point_list=[]
        for point in weatherPoints:
            point_data = weather_data[[point]]
            weather_point_list.append(point_data)
            all_weather_point = pd.concat(weather_point_list,axis=1) #axis=1, rowwise concat
            all_weather_point = all_weather_point[~all_weather_point.index.duplicated()]#To get rid of duplicated index
            all_weather_point = all_weather_point.reindex(pd.DatetimeIndex(start = all_weather_point.index[0], 
                                                                           periods=len(single_building_energy), 
                                                                           freq='H')).fillna(method='ffill').fillna(method='bfill')
#             in some cases, there are more than 1 data in the same hour, creating more than 8760 points
#             to make them consistent, take the first index minuits, based on the number of energy data,
#             transform them into hourly data. Then we get the same number of energy data (mostly8760)
#             DatatimeIndex them, reindex then is able to match and select those hour with the minuites
#             same as first index, regulating the data to be consistent with number of energy points, get
#             rid of the repeated weather data in the same hour.
    
        # Get schedule data for given building
        schedule_name = meta.T[single_building].annualschedule
        schedule_data = pd.read_csv(os.path.join('/Users/t.wang/Desktop/Dissertation/Python/input/',
                                                schedule_name),index_col=0, header=None, parse_dates=True)
        schedule_data = schedule_data.tz_localize(single_timezone, ambiguous = 'infer')
        schedule_data.columns = ['seasonal']
        schedule_data = schedule_data.reindex(pd.DatetimeIndex(start = schedule_data.index[0], periods=len(single_building_energy), 
                                                               freq='H')).fillna(method='ffill').fillna(method='bfill')
#         same trick is applied to selecting schedule data


        
        features = pd.merge(pd.DataFrame(single_building_energy.index.tz_localize(None)), 
                    schedule_data.reset_index(drop=True), right_index=True, left_index=True)#remove the time zone information
                #Map the schedule, otherwise the TimeSplits will not be able to capture all schedules, resulting in inconsistency of traning/test feature dimensions
        features['seasonal_num'] = features.seasonal.map({'Break':0, 'Regular':1, 'Holiday':2, 'Summer':3})
        features = features.drop('seasonal', axis=1)
        features = pd.concat([features, all_weather_point.reset_index(drop=True)], axis=1) #.reset_index(drop=True) to get rid of the time index, otherwise two sets data will stratify
#                 features = features.fillna(method='ffill').fillna(method='bfill')
        # features = np.array(features)
        labels = single_building_energy.values
        '''FeatureTool'''
        es = ft.EntitySet(id = 'buildingFeatures') #create Entity set
        # create an entity from feature table, unique index is created
        es = es.entity_from_dataframe(entity_id='featureData', dataframe=features,
                      make_index=True, index='feature_id', time_index = 'timestamp')
        

        features_FE, feature_names = ft.dfs(entityset = es, target_entity = 'featureData', max_depth = 2
                        ,agg_primitives = agg_primitives,
                        trans_primitives = trans_primitives, verbose = False, n_jobs=1) #Not sure why n_jobs more than 1 is not working

        # one hot encoding for categorical data
        features_enc, feature_names_enc = ft.encode_features(features_FE, feature_names)
        # Replace infinity number arising after feature generation
        features_enc = features_enc.replace(np.inf, '9999')
        features_enc = features_enc.replace(-np.inf, '-9999')
        features_enc = features_enc.replace([np.nan,''],0)
#         print(features_enc)

        '''Feature Selection'''
#                 Filter methods - Removing features with low variance - SKlearn
        y = labels
        X = features_enc
        sel = VarianceThreshold(threshold=(varianceThreshold * (1 - varianceThreshold)))
        X_Variance = sel.fit_transform(X)

#                 Filter methods - Univariate feature selection - SKlearn    
        X_Variance_Percentile = SelectPercentile(score_func=SelectPercentile_func, percentile=SelectPercentile_num).fit_transform(X_Variance,y)
#         Extract one year data for later Wrapper methods comparison
#         pd.DataFrame(X_Variance_Percentile).to_csv('/Users/t.wang/Desktop/Dissertation/Python/WT-result/'+ 'One_year_features' + '.csv', index=False)
#         pd.DataFrame(y).to_csv('/Users/t.wang/Desktop/Dissertation/Python/WT-result/'+ 'One_year_target' + '.csv', index=False)
                                
#                 Wrapper methods - Recursive feature elimination -SKlearn
        Wrapper_model = ml_Models
        rfe = RFE(estimator=Wrapper_model, step=RFE_step,verbose=True)
        rfe = rfe.fit(X_Variance_Percentile, y)
        X_Variance_Percentile_RFE = rfe.transform(X_Variance_Percentile)  
#         add timestamp back to features for further Time split
        X_Variance_Percentile_RFE = pd.DataFrame(X_Variance_Percentile_RFE).set_index(features.timestamp)
        y = pd.DataFrame(y).set_index(features.timestamp)
#         print(X_Variance_Percentile_RFE)
#         print(y.shape)
        
                # Create TimeSeriesSplit
        # get the month number for splitting
        months = np.array(single_building_energy.index.month.unique())
        tscv = TimeSeriesSplit(n_splits=n_timeSeriesSplits)
        train_test_list = []
        for train_index, test_index in tscv.split(months):
            months_train, months_test = months[train_index], months[test_index]
            train_test_list.append([months_train, months_test])
        # Add the 'every-four-month' version, 5th month is duplicated in Clayton's notebook
        train_test_list.append([np.concatenate([months[0:3],months[4:7],months[8:11]]),
                                             np.array([months[3],months[7],months[11]])])
        
        index = 0 #index for each TimeSeries cv
        for train_index, test_index in train_test_list: #get rid of the 'array', extract the numeric months from the list
        #     print(train_index, test_index)
            months_for_train = train_index
            months_for_test = test_index

            # Create features and labels under last 'for' loop such that all TimeSeriesSplit could be implenmented
            def get_features_and_labels(train_or_test):
                nonlocal X_Variance_Percentile_RFE
                nonlocal y
                X_Variance_Percentile_RFE_n = np.array(X_Variance_Percentile_RFE[X_Variance_Percentile_RFE.index.month.isin(train_or_test)])
                y_n = np.array(y[y.index.month.isin(train_or_test)].T)[0]
                #rename _n is required, otherwise the function will run on top of incomplete dataset after one running(after traindata, testdata disappeared)     
#                 transform it to array for ml 
                return X_Variance_Percentile_RFE_n, y_n


            # test on model and calculate errors
            X_train, y_train = get_features_and_labels(train_or_test=months_for_train)
            X_test, y_test = get_features_and_labels(train_or_test=months_for_test)
            print(months_for_train, X_train.shape, y_train.shape, months_for_test, X_test.shape, y_test.shape)
#             print(X_test)
#             print(y_test)
#             compare = pd.concat([pd.DataFrame(X_train), pd.DataFrame(X_test)], axis=1) 
#             pd.DataFrame(X_train).to_csv('/Users/t.wang/Desktop/' + 'X_train' +'.csv', index=False)
#             pd.DataFrame(X_test).to_csv('/Users/t.wang/Desktop/' + 'X_test' +'.csv', index=False)
            ml_Models.fit(X_train, y_train)
            predictions = ml_Models.predict(X_test)
            errors = abs(predictions - y_test)
            MAPE = 100 * np.mean((errors / y_test))
            NMBE = 100 * (sum(y_test - predictions) / (pd.Series(y_test).count() * np.mean(y_test)))
            CVRSME = 100 * ((sum((y_test - predictions)**2) / (pd.Series(y_test).count()-1))**(0.5)) / np.mean(y_test)
            RSQUARED = r2_score(y_test, predictions)
            
            index+=1
            if (buildingindex==1):
#               create the csv at the start of looping each metrics for each building
                temporary = pd.DataFrame(columns=['buildingName','MAPE','NMBE','CVRSME','RSQUARED',
                                                  'trainedMonths_'+str(months_for_train),'testMonths_'+str(months_for_test)])
                temporary.to_csv('/Users/t.wang/Desktop/Dissertation/Python/WT-result/' 
                                 + ml_Models_names + '_metrics_cross_validation_' + str(index) + '.csv', index=False)
#           read and the csv and metrics result
            metrics_prev = pd.read_csv('/Users/t.wang/Desktop/Dissertation/Python/WT-result/' 
                                 + ml_Models_names + '_metrics_cross_validation_' + str(index) + '.csv')
            df = pd.DataFrame([[single_building, MAPE, NMBE, CVRSME, RSQUARED]],
                              columns=['buildingName','MAPE','NMBE','CVRSME','RSQUARED'])
#           write the csv
            metrics = pd.concat([df, metrics_prev], sort=False)
#           export csv
            metrics.to_csv('/Users/t.wang/Desktop/Dissertation/Python/WT-result/' 
                                 + ml_Models_names + '_metrics_cross_validation_' + str(index) + '.csv', index=False)

    
ml_Models_lists = [['RandomForestRegressor_FE', RandomForestRegressor(n_estimators=100, random_state=42)]]
weatherPoints = ['TemperatureC', 'Humidity','Dew PointC','Sea Level PressurehPa', 
                 'Wind Direction','Conditions','WindDirDegrees']
n_timeSeriesSplits = 3
buildingNames = meta.dropna(subset=['annualschedule']).index[100:101] #drop buildings with missing schedule
agg_primitives = []
trans_primitives = ['weekday','hour','is_weekend','divide_numeric',
                               'and','multiply_numeric','divide_by_feature','absolute','week','subtract_numeric',
                               'percentile']
varianceThreshold = 0.8
SelectPercentile_num = 5
SelectPercentile_func = f_regression
RFE_step = 10

for elem in ml_Models_lists:
#     ml_Models_names = elem[0], ml_Models = elem[1], not sure why this gives warning 'no n_estimator'
    loopModels_and_Metrics(ml_Models_names = elem[0], ml_Models=elem[1],weatherPoints=weatherPoints,
                           buildingNames=buildingNames, n_timeSeriesSplits=n_timeSeriesSplits, 
                           agg_primitives=agg_primitives, trans_primitives=trans_primitives,varianceThreshold=varianceThreshold,
                           SelectPercentile_num=SelectPercentile_num, SelectPercentile_func=SelectPercentile_func, RFE_step=RFE_step)

end = time.time()
elapsed = end - start 
print('Time per building after FE and FS:'+ time.strftime("%H:%M:%S", time.gmtime(elapsed)))



# all_weather_point
# schedule_data
# single_building_energy
# train_test_list
# X_train,y_train
# X_train.shape,y_train.shape
# X_test,y_test
# X_test.shape,y_test.shape
# buildingNames




RandomForestRegressor_FE
_____________
Modelling:Office_Mark
Fitting estimator with 128 features.
Fitting estimator with 118 features.
Fitting estimator with 108 features.
Fitting estimator with 98 features.
Fitting estimator with 88 features.
Fitting estimator with 78 features.
Fitting estimator with 68 features.
[5 6 7] (2208, 64) (2208,) [ 8  9 10] (2208, 64) (2208,)
[ 5  6  7  8  9 10] (4416, 64) (4416,) [11 12  1] (2209, 64) (2209,)
[ 5  6  7  8  9 10 11 12  1] (6625, 64) (6625,) [2 3 4] (2135, 64) (2135,)
[ 5  6  7  9 10 11  1  2  3] (6552, 64) (6552,) [ 8 12  4] (2208, 64) (2208,)
Time per building after FE and FS:00:07:16
